In [1]:
// boring imports
var SVM = require('ml-svm');
var {loadLabelledWine} = require('./utils');
var Plot = require('plotly-notebook-js');

'use strict'

# Cross Validation

In machine learning Cross Validation is a technique / practice where we reserve a percentage of our training dataset in order to test the performance of our model.

e.g. you might have a 80/20 split using only 80% of data for training and retaining 20% to test. 

But how to decide what to retain?



## K Fold Validation

We can increase the reliablility of our metrics and still evaluate performance across the whole training dataset by using k-fold cross validation.

Essentially, we will do cross validation K times *drawing* a different cross validation set each time from the training data.

Luckily we still have a library to help, here are the [docs](https://mljs.github.io/cross-validation/)

In [2]:
var crossValidation = require('ml-cross-validation');
var SVM = require('ml-svm');

'use strict'

In [14]:
var {features, dataset} = loadLabelledWine({verbose: true});

var inputs = dataset.map(d => d.slice(0,5))
var labels = dataset.map(d => d[13]-2) // remember the domain mapping

our dataset has 178 rows and  14 columns
Alcohol | Malic Acid | Ash | Alcalinity of ash | Magnesium | Total phenols | Flavanoids | Nonflavanoid phenols | Proanthocyanins | Color intensity | Hue | OD280/OD315 of diluted wines | Proline | Class
---------------------------
14.23 | 1.71 | 2.43 | 15.6 | 127 | 2.8  | 3.06 | 0.28 | 2.29 | 5.64 | 1.04 | 3.92 | 1065 | 1
 13.2 | 1.78 | 2.14 | 11.2 | 100 | 2.65 | 2.76 | 0.26 | 1.28 | 4.38 | 1.05 | 3.4  | 1050 | 1
13.16 | 2.36 | 2.67 | 18.6 | 101 | 2.8  | 3.24 | 0.3  | 2.81 | 5.68 | 1.03 | 3.17 | 1185 | 1
14.37 | 1.95 | 2.5  | 16.8 | 113 | 3.85 | 3.49 | 0.24 | 2.18 | 7.8  | 0.86 | 3.45 | 1480 | 1
13.24 | 2.59 | 2.87 | 21   | 118 | 2.8  | 2.69 | 0.39 | 1.82 | 4.32 | 1.04 | 2.93 | 735  | 1


The module creates a confusion matri for us whih we can query for metrics

In [15]:
var options = {
  C: 1,  // Regularisation Parameter
  tol: 10e-4,  // numerical tolerance
  maxPasses: 10,  // max number of times to iterate over alphas when no change
  maxIterations: 10000,  // max number of iterations
  kernel: 'linear', // kernel to use see full like in kernel docs
  kernelOptions: {
    sigma: 0.5  // sets the gaussian width
  }
};

var confusionMatrix = crossValidation.kFold(SVM, inputs, labels, options, 20);

'use strict'

In [19]:
var KNN = require('ml-knn');

var options = { k: 25 };

var confusionMatrix = crossValidation.kFold(KNN, inputs, labels, options, 20);

'use strict'

In [20]:
var accuracy = confusionMatrix.getAccuracy();
console.log("Accuracy", accuracy)
console.log("F1 -1", confusionMatrix.getF1Score(-1))
console.log("F1 0", confusionMatrix.getF1Score(0))
console.log("F1 1", confusionMatrix.getF1Score(1))


var M = confusionMatrix.getMatrix();

var trace = { 
    x: [0,1,2],
    y: [0,1,2],
    z: M,
    type: 'heatmap',
    showscale: false,
    colorscale:[[0, '#3D9970'], [100, '#001f3f']]
};

var annotations = [];

M.map((a,y) => {
    a.map((b,x) => {
        annotations.push(
            {
                x: x,
                y: y,
                text: M[y][x],
                font: {
                    family: 'Arial',
                    size: 12,
                    color: 'white'
                  },
                showarrow: false
            }
        )
    })
})

var layout = { 
    xaxis: { title: "predicted", side: 'top' },
    yaxis: { title: "actuals", nticks: 6, autosize: false, autorange: 'reversed' },
    annotations,
    width: 500, height: 500};

$$html$$ = Plot.createPlot([trace], layout).render();


Accuracy 0.625
F1 -1 0.6962962962962963
F1 0 0.6554621848739496
F1 1 0.42424242424242425


We should look at average f1 scores over classes too